In [93]:
import pandas as pd

In [94]:
# df = pd.read_csv('/Users/qingtan/Desktop/discovery_mod/bpic/bpic_2015_1/_log_preprocessed_not_filtered.csv')
df = pd.read_csv('/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_2020_gt/autonomous/train_preprocessed.csv')
print(df)

       Unnamed: 0  case_id                timestamp        resource  \
0           29298      423  2018/02/28 22:10:26.000        EMPLOYEE   
1           29299      423  2018/02/28 22:10:29.000  ADMINISTRATION   
2           29300      423  2018/03/05 23:20:50.000    BUDGET OWNER   
3           29301      423  2018/03/12 15:15:50.000      SUPERVISOR   
4           29302      423  2018/04/10 08:00:00.000        EMPLOYEE   
...           ...      ...                      ...             ...   
70053       12157    86541  2017/12/20 23:58:57.000        EMPLOYEE   
70054       12158    86541  2017/12/20 23:59:25.000    PRE_APPROVER   
70055       12159    86541  2017/12/21 02:37:34.000      SUPERVISOR   
70056       12160    86541  2017/12/22 00:28:03.000          SYSTEM   
70057       12161    86541  2018/01/05 03:31:14.000          SYSTEM   

                    activity_name                    agent_activity_type  \
0                Permit SUBMITTED              EMPLOYEE|Permit SUBMITTE

In [95]:
# get the basic information from the table
print(f"case num: {df['case_id'].nunique()}")
print(f"activity num: {df['activity_name'].nunique()}")
print(f"agent num: {df['agent'].nunique()}")

df_copy = df.copy()

df_copy['timestamp'] = pd.to_datetime(df_copy['start_timestamp'])

print(f"largest period: {df_copy['timestamp'].max() - df_copy['timestamp'].min()}")

case num: 5563
activity num: 21
agent num: 8
largest period: 772 days 10:22:17


In [96]:
from collections import Counter
from itertools import combinations
import pandas as pd

# Group agents by case_id
case_agents = df.groupby("case_id")["agent"].unique()
case_agents


case_id
423      [0, 6, 7, 1, 5]
438         [0, 6, 1, 5]
445         [0, 6, 1, 5]
455               [0, 6]
461         [0, 6, 1, 5]
              ...       
86493       [0, 2, 1, 5]
86514       [0, 1, 4, 5]
86525          [0, 1, 5]
86533          [0, 1, 5]
86541       [0, 2, 1, 5]
Name: agent, Length: 5563, dtype: object

In [97]:
# Assuming case_agents is a dictionary or Series: {case_id: [agent1, agent2, ...]}
# Normalize each agent list into a sorted tuple
group_counts = Counter()

for agents in case_agents.values:
    normalized_group = tuple(sorted(agents))  # Sorting to make ['a','b'] and ['b','a'] identical
    group_counts[normalized_group] += 1

# Convert to a dict if needed
group_counts_dict = dict(group_counts)

# Print example
filtered_group_information = {}
filtered_group_dict = {}
ini = 1
count_sum = 0
for group, count in group_counts_dict.items():
    if count >= 0.01 * len(case_agents):
        count_sum += count
        group_name = 'G' + str(ini)
        filtered_group_information[group] = group_name
        filtered_group_dict[group] = count
        print(group, ":", count)
        ini += 1

print(count_sum)

(0, 1, 5, 6, 7) : 1374
(0, 1, 5, 6) : 2328
(0, 1, 3, 5, 6, 7) : 159
(0, 1, 3, 5, 6) : 220
(0, 1, 5) : 505
(0, 1, 2, 5) : 485
(0, 1, 3, 5) : 115
(0, 1, 2, 4, 5) : 56
5242


In [98]:
from collections import defaultdict


# Ensure all agent lists are sorted tuples for consistent matching
normalized_case_agents = case_agents.apply(lambda agents: tuple(sorted(agents)))

# Invert: from group → list of case_ids
group_to_cases = defaultdict(list)
filtered_group_case_info = defaultdict(list)

for case_id, group in normalized_case_agents.items():
    if group in filtered_group_dict:
        group_to_cases[group].append(case_id)
        filtered_group_case_info[filtered_group_information[group]].append(case_id)


# Convert to a plain dictionary if needed
group_to_cases_dict = dict(group_to_cases)


In [99]:
from collections import defaultdict

# Start a nested dictionary
group_agent_activity_freq = {}

for group, case_ids in group_to_cases_dict.items():
    # Filter log by case_ids for this group
    df_group = df[df['case_id'].isin(case_ids)]

    # Filter again to keep only events done by agents in the group
    df_group = df_group[df_group['agent'].isin(group)]

    # Initialize nested structure
    agent_activity_counts = defaultdict(lambda: defaultdict(int))

    for _, row in df_group.iterrows():
        agent = row['agent']
        activity = row['activity_name']
        agent_activity_counts[agent][activity] += 1

    # Convert to normal dict for export
    group_agent_activity_freq[group] = {
        agent: dict(activities)
        for agent, activities in agent_activity_counts.items()
    }

group_agent_activity_freq

{(0,
  1,
  5,
  6,
  7): {0: {'Permit SUBMITTED': 1460,
   'Start trip': 1374,
   'End trip': 1374,
   'Declaration SUBMITTED': 1791,
   'Permit REJECTED': 84,
   'Request For Payment SUBMITTED': 431,
   'Declaration REJECTED': 500,
   'Declaration SAVED': 10,
   'Request For Payment REJECTED': 52,
   'Request For Payment SAVED': 6}, 6: {'Permit APPROVED': 1410,
   'Declaration APPROVED': 1323,
   'Request For Payment APPROVED': 377,
   'Declaration REJECTED': 464,
   'Request For Payment REJECTED': 50,
   'Permit REJECTED': 36}, 7: {'Permit APPROVED': 1306,
   'Declaration APPROVED': 1234,
   'Request For Payment APPROVED': 354,
   'Declaration REJECTED': 25,
   'Permit REJECTED': 21,
   'Request For Payment REJECTED': 2}, 1: {'Permit FINAL_APPROVED': 1374,
   'Declaration FINAL_APPROVED': 1281,
   'Permit REJECTED': 28,
   'Request For Payment FINAL_APPROVED': 377,
   'Declaration REJECTED': 18,
   'Request For Payment REJECTED': 1}, 5: {'Request Payment': 1658,
   'Payment Handled'

In [100]:
import math

# activity matrix
def get_activity_matrix(diff_task_list, diff_resource_list, rec_act_freq_dic):
    # activity_matrix_dic = {agent_1: [0 ,4, 8, ...], agent_2: [...], ...} 
    # elements in [0, 4, 8, ...] is the same task order as in diff_task_list
    activity_matrix_dic = {}
    for resource in diff_resource_list:
        task_freq_list = []
        for task in diff_task_list:
            # if (task, resource) in frequency_map:
            #     task_freq_list.append(frequency_map[(task, resource)])
            if task in rec_act_freq_dic[resource].keys():
                task_freq_list.append(rec_act_freq_dic[resource][task])
            else:
                task_freq_list.append(0)
        activity_matrix_dic[resource] = task_freq_list
    return activity_matrix_dic

def get_correlation_value(activity_matrix, first_resource, second_resource):
    x_list = activity_matrix[first_resource]
    y_list = activity_matrix[second_resource]
    # frac = (nominator_1 - nominator_2) / (denominator_1 * denominator_2)
    n = len(x_list)
    nominator_2 = sum(x_list) * sum(y_list)
    sum_x_times_y = 0
    sum_x_square = 0
    sum_y_square = 0
    for index in range(len(x_list)):
        x = x_list[index]
        y = y_list[index]
        sum_x_times_y += (x * y)
        sum_x_square += x * x
        sum_y_square += y * y
    nominator_1 = n * sum_x_times_y
    denominator_1 = math.sqrt(n * sum_x_square - sum(x_list) * sum(x_list))
    denominator_2 = math.sqrt(n * sum_y_square - sum(y_list) * sum(y_list))
    if denominator_1 * denominator_2 == 0:
        return 0
    else:
        correlation_value = (nominator_1 - nominator_2) / (denominator_1 * denominator_2)
        return correlation_value

def get_relation_tuple_list(activity_matrix, t1):
    # relation_tuple_list = [(agent_1, agent_2), (agent_1, agent_3), ...]
    relation_tuple_list = []
    for first_resource in activity_matrix:
        for second_resource in activity_matrix:
            if first_resource != second_resource:
                correlation_value = get_correlation_value(activity_matrix, first_resource, second_resource)
                if correlation_value >= t1:
                    if (first_resource, second_resource) not in relation_tuple_list:
                        relation_tuple_list.append((first_resource, second_resource))
    return relation_tuple_list

def get_resource_clusters_list(relation_tuple_list, diff_resource_list):
    completed_resource_list = []
    resource_clusters_list = []
    for resource in diff_resource_list:
        if resource not in completed_resource_list:
            resource_cluster = []
            resource_cluster.append(resource)
            new_cluster_len = len(resource_cluster)
            cluster_len = -1
            while (new_cluster_len != cluster_len):
                cluster_len = new_cluster_len
                for relation_tuple in relation_tuple_list:
                    resource_1 = relation_tuple[0]
                    resource_2 = relation_tuple[1]
                    if resource_1 in resource_cluster or resource_2 in resource_cluster:
                        if resource_1 not in resource_cluster:
                            resource_cluster.append(resource_1)
                        if resource_2 not in resource_cluster:
                            resource_cluster.append(resource_2)
                new_cluster_len = len(resource_cluster)
            resource_clusters_list.append(resource_cluster)
            for completed_resource in resource_cluster:
                if completed_resource not in completed_resource_list:
                    completed_resource_list.append(completed_resource)
    return resource_clusters_list


def get_formal_cluster_info(cluster_agent_list):
    role_dic = {}
    ini = 1
    for cluster in cluster_agent_list:
        role_name = 'GR' + str(ini)
        role_dic[role_name] = cluster
        ini += 1

    return role_dic

In [101]:
print(filtered_group_information)
print(filtered_group_case_info)

{(0, 1, 5, 6, 7): 'G1', (0, 1, 5, 6): 'G2', (0, 1, 3, 5, 6, 7): 'G3', (0, 1, 3, 5, 6): 'G4', (0, 1, 5): 'G5', (0, 1, 2, 5): 'G6', (0, 1, 3, 5): 'G7', (0, 1, 2, 4, 5): 'G8'}
defaultdict(<class 'list'>, {'G1': [423, 539, 563, 682, 720, 759, 953, 978, 1050, 1076, 1154, 1239, 1284, 1311, 1325, 1342, 1432, 1444, 1480, 1523, 1552, 1591, 1614, 1654, 1714, 1720, 1753, 1863, 1882, 1920, 1954, 1967, 1992, 2005, 2017, 2033, 2053, 2082, 2111, 2125, 2137, 2152, 2208, 2306, 2399, 2481, 2558, 2598, 2615, 2628, 2697, 2830, 2881, 2919, 2974, 2988, 3003, 3028, 3098, 3152, 3211, 3218, 3224, 3250, 3265, 3281, 3288, 3422, 3477, 3540, 3592, 3613, 3625, 3648, 3724, 3793, 3823, 3914, 3994, 4007, 4019, 4071, 4168, 4181, 4292, 4337, 4364, 4398, 4499, 4514, 4600, 4651, 4664, 4714, 4756, 4770, 4807, 4879, 4930, 4969, 5016, 5050, 5188, 5204, 5244, 5314, 5335, 5368, 5380, 5487, 5505, 5511, 5581, 5648, 5723, 5735, 5832, 5847, 5938, 5971, 6025, 6076, 6125, 6186, 6275, 6295, 6361, 6373, 6435, 6517, 6548, 6567, 6579, 6

In [102]:
# apply activity matrix to generate roles
group_role_info_dic = {}

for group in group_agent_activity_freq.keys():
    diff_resource_list = list(group)
    diff_task_list = sorted({activity for agent_dict in group_agent_activity_freq[group].values() for activity in agent_dict})
    activity_matrix = get_activity_matrix(diff_task_list, diff_resource_list, group_agent_activity_freq[group])
    relation_tuple_list = get_relation_tuple_list(activity_matrix, 0.98)
    cluster_agent_list = get_resource_clusters_list(relation_tuple_list, diff_resource_list)
    role_dic = get_formal_cluster_info(cluster_agent_list)
    group_role_info_dic[filtered_group_information[group]] = role_dic

print(group_role_info_dic)

{'G1': {'GR1': [0], 'GR2': [1], 'GR3': [5], 'GR4': [6], 'GR5': [7]}, 'G2': {'GR1': [0], 'GR2': [1], 'GR3': [5], 'GR4': [6]}, 'G3': {'GR1': [0], 'GR2': [1], 'GR3': [3], 'GR4': [5], 'GR5': [6], 'GR6': [7]}, 'G4': {'GR1': [0], 'GR2': [1], 'GR3': [3], 'GR4': [5], 'GR5': [6]}, 'G5': {'GR1': [0], 'GR2': [1], 'GR3': [5]}, 'G6': {'GR1': [0], 'GR2': [1], 'GR3': [2], 'GR4': [5]}, 'G7': {'GR1': [0], 'GR2': [1], 'GR3': [3], 'GR4': [5]}, 'G8': {'GR1': [0], 'GR2': [1], 'GR3': [2], 'GR4': [4], 'GR5': [5]}}


In [103]:
from collections import defaultdict

# Final result: {G: {GR: {activity: count}}}
group_role_activity_counts = {}

for group, roles in group_role_info_dic.items():
    group_role_activity_counts[group] = {}
    
    for role, agents in roles.items():
        # Filter df to only those agents in this role
        role_df = df[df['agent'].isin(agents)]
        
        # Group by activity_type and count occurrences
        activity_counts = (
            role_df['activity_name']
            .value_counts()
            .to_dict()
        )
        group_role_activity_counts[group][role] = activity_counts


print(group_role_activity_counts)


{'G1': {'GR1': {'Declaration SUBMITTED': 6480, 'Permit SUBMITTED': 5775, 'Start trip': 5563, 'End trip': 5563, 'Request For Payment SUBMITTED': 1659, 'Declaration REJECTED': 1410, 'Permit REJECTED': 269, 'Request For Payment REJECTED': 188, 'Declaration SAVED': 58, 'Request For Payment SAVED': 14, 'Permit SAVED': 13}, 'GR2': {'Permit FINAL_APPROVED': 4970, 'Declaration FINAL_APPROVED': 4856, 'Request For Payment FINAL_APPROVED': 1437, 'Permit APPROVED': 534, 'Declaration APPROVED': 195, 'Permit REJECTED': 104, 'Declaration REJECTED': 97, 'Request For Payment APPROVED': 29, 'Request For Payment REJECTED': 9}, 'GR3': {'Request Payment': 6410, 'Payment Handled': 6409, 'Send Reminder': 1406}, 'GR4': {'Permit APPROVED': 4195, 'Declaration APPROVED': 3945, 'Declaration REJECTED': 1219, 'Request For Payment APPROVED': 1194, 'Request For Payment REJECTED': 163, 'Permit REJECTED': 98, 'Permit FOR_APPROVAL': 1}, 'GR5': {'Permit APPROVED': 1496, 'Declaration APPROVED': 1389, 'Request For Payment 

In [104]:
# cluster similar roles
diff_task_list_1, diff_resource_list_1 = [], []
for group_id in group_role_activity_counts.keys():
    for group_role_id in group_role_activity_counts[group_id].keys():
        diff_resource_list_1.append((group_id, group_role_id))
        for act in group_role_activity_counts[group_id][group_role_id].keys():
            if act not in diff_task_list_1:
                diff_task_list_1.append(act)


group_role_act_freq_dic = {}
for group_id in group_role_activity_counts.keys():
    for group_role_id in group_role_activity_counts[group_id].keys():
        group_role_act_freq_dic[(group_id, group_role_id)] = group_role_activity_counts[group_id][group_role_id]


def get_formal_role_info(cluster_agent_list):
    role_dic = {}
    ini = 1
    for cluster in cluster_agent_list:
        role_name = 'Role' + str(ini)
        role_dic[role_name] = cluster
        ini += 1

    return role_dic


activity_matrix_1 = get_activity_matrix(diff_task_list_1, diff_resource_list_1, group_role_act_freq_dic)
relation_tuple_list_1 = get_relation_tuple_list(activity_matrix_1, 0.999)
cluster_agent_list_1 = get_resource_clusters_list(relation_tuple_list_1, diff_resource_list_1)
role_dic_1 = get_formal_role_info(cluster_agent_list_1)

print(role_dic_1)

{'Role1': [('G1', 'GR1'), ('G2', 'GR1'), ('G3', 'GR1'), ('G4', 'GR1'), ('G5', 'GR1'), ('G6', 'GR1'), ('G7', 'GR1'), ('G8', 'GR1')], 'Role2': [('G1', 'GR2'), ('G2', 'GR2'), ('G3', 'GR2'), ('G4', 'GR2'), ('G5', 'GR2'), ('G6', 'GR2'), ('G7', 'GR2'), ('G8', 'GR2')], 'Role3': [('G1', 'GR3'), ('G2', 'GR3'), ('G3', 'GR4'), ('G4', 'GR4'), ('G5', 'GR3'), ('G6', 'GR4'), ('G7', 'GR4'), ('G8', 'GR5')], 'Role4': [('G1', 'GR4'), ('G2', 'GR4'), ('G3', 'GR5'), ('G4', 'GR5')], 'Role5': [('G1', 'GR5'), ('G3', 'GR6')], 'Role6': [('G3', 'GR3'), ('G4', 'GR3'), ('G7', 'GR3')], 'Role7': [('G6', 'GR3'), ('G8', 'GR3')], 'Role8': [('G8', 'GR4')]}


In [105]:
# process the results (role_dic_1)
final_group_role_agent_dic = {}

for true_role in role_dic_1.keys():
    for group_role_tuple in role_dic_1[true_role]:
        group_id, group_role_id = group_role_tuple
        if group_id not in final_group_role_agent_dic.keys():
            final_group_role_agent_dic[group_id] = {}
            final_group_role_agent_dic[group_id][true_role] = group_role_info_dic[group_id][group_role_id]
        else:
            final_group_role_agent_dic[group_id][true_role] = group_role_info_dic[group_id][group_role_id]

print(final_group_role_agent_dic)


{'G1': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role5': [7]}, 'G2': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6]}, 'G3': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role5': [7], 'Role6': [3]}, 'G4': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role6': [3]}, 'G5': {'Role1': [0], 'Role2': [1], 'Role3': [5]}, 'G6': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role7': [2]}, 'G7': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role6': [3]}, 'G8': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role7': [2], 'Role8': [4]}}


In [106]:
filtered_group_case_info_dic = dict(filtered_group_case_info)

print(final_group_role_agent_dic)
print(filtered_group_case_info_dic)

{'G1': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role5': [7]}, 'G2': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6]}, 'G3': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role5': [7], 'Role6': [3]}, 'G4': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role4': [6], 'Role6': [3]}, 'G5': {'Role1': [0], 'Role2': [1], 'Role3': [5]}, 'G6': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role7': [2]}, 'G7': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role6': [3]}, 'G8': {'Role1': [0], 'Role2': [1], 'Role3': [5], 'Role7': [2], 'Role8': [4]}}
{'G1': [423, 539, 563, 682, 720, 759, 953, 978, 1050, 1076, 1154, 1239, 1284, 1311, 1325, 1342, 1432, 1444, 1480, 1523, 1552, 1591, 1614, 1654, 1714, 1720, 1753, 1863, 1882, 1920, 1954, 1967, 1992, 2005, 2017, 2033, 2053, 2082, 2111, 2125, 2137, 2152, 2208, 2306, 2399, 2481, 2558, 2598, 2615, 2628, 2697, 2830, 2881, 2919, 2974, 2988, 3003, 3028, 3098, 3152, 3211, 3218, 3224, 3250, 3265, 3281, 3288, 3422, 3477, 3540, 3592, 3613,

In [107]:
import pandas as pd

def enrich_events_df(events_df, group_case_info, group_role_info):
    # Step 1: Build a reverse map from agent to list of (group_id, role)
    resource_to_group_roles = {}
    for group_id, role_dict in group_role_info.items():
        for role_id, agent_list in role_dict.items():
            for agent in agent_list:
                if agent not in resource_to_group_roles:
                    resource_to_group_roles[agent] = []
                resource_to_group_roles[agent].append((group_id, role_id))
    
    # Step 2: Function to match and assign group/role to an event
    def match_group_role(row):
        resource_id = row['agent']
        case_id = row['case_id']
        
        matches = resource_to_group_roles.get(resource_id, [])
        for group_id, role_id in matches:
            if case_id in group_case_info.get(group_id, []):
                return pd.Series({'group_id': group_id, 'group_role_id': role_id})
        return pd.Series({'group_id': None, 'group_role_id': None})
    
    # Step 3: Apply the matching logic
    matched = events_df.apply(match_group_role, axis=1)
    
    # Step 4: Merge results back and drop unmatched rows
    enriched_df = pd.concat([events_df, matched], axis=1)
    enriched_df = enriched_df.dropna(subset=['group_id', 'group_role_id']).reset_index(drop=True)
    
    return enriched_df


new_event_df = enrich_events_df(df, filtered_group_case_info_dic, final_group_role_agent_dic)
new_event_df.to_csv('/Users/qingtan/Desktop/role_mining_code/sample_new_events_2020_train.csv')
print(new_event_df)

       Unnamed: 0  case_id                timestamp        resource  \
0           29298      423  2018/02/28 22:10:26.000        EMPLOYEE   
1           29299      423  2018/02/28 22:10:29.000  ADMINISTRATION   
2           29300      423  2018/03/05 23:20:50.000    BUDGET OWNER   
3           29301      423  2018/03/12 15:15:50.000      SUPERVISOR   
4           29302      423  2018/04/10 08:00:00.000        EMPLOYEE   
...           ...      ...                      ...             ...   
67155       12157    86541  2017/12/20 23:58:57.000        EMPLOYEE   
67156       12158    86541  2017/12/20 23:59:25.000    PRE_APPROVER   
67157       12159    86541  2017/12/21 02:37:34.000      SUPERVISOR   
67158       12160    86541  2017/12/22 00:28:03.000          SYSTEM   
67159       12161    86541  2018/01/05 03:31:14.000          SYSTEM   

                    activity_name                    agent_activity_type  \
0                Permit SUBMITTED              EMPLOYEE|Permit SUBMITTE

In [60]:
# Step 1: Reverse mapping: role set → groups
role_set_to_groups = defaultdict(list)

for group_id, roles_dict in final_group_role_agent_dic.items():
    role_set = tuple(sorted(roles_dict.keys()))  # sort to ensure consistency
    role_set_to_groups[role_set].append(group_id)

# Convert defaultdict to dict if needed
role_set_to_groups = dict(role_set_to_groups)

# Print result
print(role_set_to_groups)

{('Role1',): ['G1', 'G13', 'G4', 'G15', 'G20', 'G3', 'G19', 'G18', 'G6', 'G7', 'G10', 'G14'], ('Role1', 'Role5'): ['G16'], ('Role1', 'Role2', 'Role3', 'Role7'): ['G21'], ('Role1', 'Role6'): ['G17'], ('Role1', 'Role2'): ['G2', 'G5'], ('Role1', 'Role3'): ['G9'], ('Role1', 'Role4'): ['G12'], ('Role2', 'Role3'): ['G8'], ('Role3',): ['G11']}


In [61]:
import pandas as pd

# Input dictionary

# Step 1: Extract all unique resources, groups, and roles
resources = set()
groups = sorted(final_group_role_agent_dic.keys())  # sort groups
roles = set()

for group, role_dict in final_group_role_agent_dic.items():
    for role, res_list in role_dict.items():
        roles.add(role)
        resources.update(res_list)

roles = sorted(roles)  # sort roles

# Step 2: Generate a DataFrame per resource
resource_role_group_matrices = {}

for resource in sorted(resources):
    data = []
    for group in groups:
        row = {}
        for role in roles:
            res_list = final_group_role_agent_dic.get(group, {}).get(role, [])
            row[role] = int(resource in res_list)
        data.append(row)
    df = pd.DataFrame(data, index=groups)
    
    # Remove rows and columns with all zeros
    df = df.loc[~(df == 0).all(axis=1)]  # remove all-zero rows
    df = df.loc[:, ~(df == 0).all(axis=0)]  # remove all-zero columns
    
    resource_role_group_matrices[resource] = df


# Step 3: Print matrices
for resource, df in resource_role_group_matrices.items():
    print(f"\nagent: {resource}")
    print(df)



agent: r11744364
     Role1
G10      1

agent: r2670601
     Role1  Role2  Role5
G1       1      0      0
G13      1      0      0
G16      0      0      1
G2       0      1      0
G21      0      1      0
G3       1      0      0
G4       1      0      0
G5       0      1      0
G8       0      1      0

agent: r3273854
     Role1  Role5  Role6
G15      1      0      0
G16      0      1      0
G17      0      0      1
G19      1      0      0
G20      1      0      0

agent: r560462
     Role1
G18      1

agent: r560872
     Role1
G10      1
G12      1
G16      1
G17      1
G18      1
G19      1
G2       1
G21      1
G3       1
G5       1
G6       1
G7       1
G9       1

agent: r560890
     Role1  Role7
G13      1      0
G15      1      0
G16      1      0
G19      1      0
G20      1      0
G21      0      1
G3       1      0
G4       1      0
G6       1      0

agent: r560912
     Role1  Role3
G1       1      0
G10      1      0
G11      0      1
G2       1      0
G20      1      

In [62]:
# 2025.08.26
# 1. consider about getting the agent model (with roles and activities)
# 2. the idea is that we re-design the case by (day / week / month) of the activity the agent involves
import pandas as pd
import os

sample_df = pd.read_csv('/Users/qingtan/Desktop/role_mining_code/sample_new_events.csv')
sample_df["role_activity"] = sample_df["group_role_id"] + "|" + sample_df["activity_type"]

output_dir = "logs_by_agent"
os.makedirs(output_dir, exist_ok=True)

# Split by agent
for agent, sub_df in sample_df.groupby("agent_id"):
    # Sort by timestamp
    sub_df["timestamp"] = pd.to_datetime(sub_df["timestamp"], errors="coerce")
    sub_df = sub_df.sort_values("timestamp").copy()
    
    # Case id = same day means same case
    day_groups = sub_df["timestamp"].dt.date
    mapping = {day: f"c{i+1}" for i, day in enumerate(sorted(day_groups.unique()))}
    sub_df["case_id"] = day_groups.map(mapping)
    
    # Save to CSV
    filename = os.path.join(output_dir, f"day_{agent}.csv")
    sub_df.to_csv(filename, index=False)
    print(f"Saved {filename}")

Saved logs_by_agent/day_r11744364.csv
Saved logs_by_agent/day_r2670601.csv
Saved logs_by_agent/day_r3273854.csv
Saved logs_by_agent/day_r560462.csv
Saved logs_by_agent/day_r560872.csv
Saved logs_by_agent/day_r560890.csv
Saved logs_by_agent/day_r560912.csv
Saved logs_by_agent/day_r560925.csv
Saved logs_by_agent/day_r560950.csv
Saved logs_by_agent/day_r9264148.csv


In [63]:
import pm4py
from pm4py.convert import convert_to_petri_net
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import networkx as nx
from discovery_AM.new_picture import graphviz_visualization_IM as viz_3
import importlib


csv_path = '/Users/qingtan/Desktop/role_mining_code/logs_by_agent/agent_day_behaviors/day_r2670601.csv'
agent_name = csv_path.split("/day_")[1].split(".csv")[0]
log_df_test = pd.read_csv(csv_path)
log_df = pm4py.format_dataframe(log_df_test, case_id='case_id', activity_key='role_activity', timestamp_key='timestamp')
log_pm4py = pm4py.convert_to_event_log(log_df)

def discover_pn_imf_pm4py(log_pm4py,im_variant,noise_threshold=0.0):
    # possible values for im_variant:
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMd
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMf
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM_CLEAN (used in pm4py.discover_petri_net_inductive)
    parameters = pm4py.utils.get_properties(log_pm4py)
    # parameters[im_variant.value.Parameters.NOISE_THRESHOLD] = noise_threshold (change)
    parameters[pm4py.algo.discovery.inductive.variants.imf.IMFParameters.NOISE_THRESHOLD] = noise_threshold
    # pt = pm4py.algo.discovery.inductive.algorithm.apply_tree(log_pm4py, variant=im_variant, parameters=parameters) (change)
    pt = pm4py.algo.discovery.inductive.algorithm.apply(log_pm4py, variant=im_variant, parameters=parameters)
    # from pm4py.convert import convert_to_petri_net
    # convert_to_petri_net(pt) will generate three parts, which are PetriNet, the initial marking and the final marking
    return convert_to_petri_net(pt)


def viz_pn(pn_pm4py,pn_im_pm4py,pn_fm_pm4py,pn_file_name_base):
    # pn_nx = pn_pm4py2nx(pn_pm4py,initial_marking=pn_im_pm4py,final_marking=pn_fm_pm4py)
    # nx.write_gml(pn_nx,pn_file_name_base+".gml")
    gviz = pn_visualizer.apply(pn_pm4py, pn_im_pm4py, pn_fm_pm4py, parameters={pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "pdf"}, variant=pn_visualizer.Variants.FREQUENCY)
    pn_visualizer.save(gviz, pn_file_name_base+".pdf")


pn_file_name_base = f'/Users/qingtan/Desktop/role_mining_code/sample_log_results/{agent_name}'
std_pn_pm4py, std_im_pm4py, std_fm_pm4py = discover_pn_imf_pm4py(log_pm4py, pm4py.algo.discovery.inductive.algorithm.Variants.IMf, noise_threshold=0.9)
viz_pn(std_pn_pm4py, std_im_pm4py, std_fm_pm4py, pn_file_name_base)
no_use = viz_3(std_pn_pm4py, std_im_pm4py, std_fm_pm4py)

/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/

In [64]:
# consider the week

import pandas as pd
import os

sample_df = pd.read_csv('/Users/qingtan/Desktop/role_mining_code/sample_new_events.csv')
sample_df["role_activity"] = sample_df["group_role_id"] + "|" + sample_df["activity_type"]

output_dir = "logs_by_agent"
os.makedirs(output_dir, exist_ok=True)

# Split by agent
for agent, sub_df in sample_df.groupby("agent_id"):
    # Sort by timestamp
    sub_df["timestamp"] = pd.to_datetime(sub_df["timestamp"], errors="coerce")
    sub_df = sub_df.sort_values("timestamp").copy()
    
    # --- case id (weekly, sequential labels c1, c2, ...) ---
    # ensure timestamp is datetime
    # df['timestamp'] = pd.to_datetime(df['timestamp'])
    # calculate week start (Monday) for each event
    sub_df['week_start'] = sub_df['timestamp'] - pd.to_timedelta(sub_df['timestamp'].dt.weekday, unit='d')

    # temporary case key: agent + week_start
    sub_df['temp_case_key'] = sub_df['week_start'].dt.strftime('%Y-%m-%d')

    # map each unique temp_case_key to c1, c2, c3, ...
    unique_cases = sub_df['temp_case_key'].unique()
    case_map = {key: f'c{i+1}' for i, key in enumerate(unique_cases)}
    sub_df['case_id'] = sub_df['temp_case_key'].map(case_map)
    
    # Save to CSV
    filename = os.path.join(output_dir, f"week_{agent}.csv")
    sub_df.to_csv(filename, index=False)
    print(f"Saved {filename}")

Saved logs_by_agent/week_r11744364.csv
Saved logs_by_agent/week_r2670601.csv
Saved logs_by_agent/week_r3273854.csv
Saved logs_by_agent/week_r560462.csv
Saved logs_by_agent/week_r560872.csv
Saved logs_by_agent/week_r560890.csv
Saved logs_by_agent/week_r560912.csv
Saved logs_by_agent/week_r560925.csv
Saved logs_by_agent/week_r560950.csv
Saved logs_by_agent/week_r9264148.csv


In [65]:
import pm4py
from pm4py.convert import convert_to_petri_net
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import networkx as nx
from discovery_AM.new_picture import graphviz_visualization_IM as viz_3
import importlib


csv_path = '/Users/qingtan/Desktop/role_mining_code/logs_by_agent/agent_week_behaviors/week_r2670601.csv'
agent_name = csv_path.split("/week_")[1].split(".csv")[0]
log_df_test = pd.read_csv(csv_path)
log_df = pm4py.format_dataframe(log_df_test, case_id='case_id', activity_key='role_activity', timestamp_key='timestamp')
log_pm4py = pm4py.convert_to_event_log(log_df)

def discover_pn_imf_pm4py(log_pm4py,im_variant,noise_threshold=0.0):
    # possible values for im_variant:
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMd
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMf
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM_CLEAN (used in pm4py.discover_petri_net_inductive)
    parameters = pm4py.utils.get_properties(log_pm4py)
    # parameters[im_variant.value.Parameters.NOISE_THRESHOLD] = noise_threshold (change)
    parameters[pm4py.algo.discovery.inductive.variants.imf.IMFParameters.NOISE_THRESHOLD] = noise_threshold
    # pt = pm4py.algo.discovery.inductive.algorithm.apply_tree(log_pm4py, variant=im_variant, parameters=parameters) (change)
    pt = pm4py.algo.discovery.inductive.algorithm.apply(log_pm4py, variant=im_variant, parameters=parameters)
    # from pm4py.convert import convert_to_petri_net
    # convert_to_petri_net(pt) will generate three parts, which are PetriNet, the initial marking and the final marking
    return convert_to_petri_net(pt)


def viz_pn(pn_pm4py,pn_im_pm4py,pn_fm_pm4py,pn_file_name_base):
    # pn_nx = pn_pm4py2nx(pn_pm4py,initial_marking=pn_im_pm4py,final_marking=pn_fm_pm4py)
    # nx.write_gml(pn_nx,pn_file_name_base+".gml")
    gviz = pn_visualizer.apply(pn_pm4py, pn_im_pm4py, pn_fm_pm4py, parameters={pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "pdf"}, variant=pn_visualizer.Variants.FREQUENCY)
    pn_visualizer.save(gviz, pn_file_name_base+".pdf")


pn_file_name_base = f'/Users/qingtan/Desktop/role_mining_code/sample_log_results/{agent_name}'
std_pn_pm4py, std_im_pm4py, std_fm_pm4py = discover_pn_imf_pm4py(log_pm4py, pm4py.algo.discovery.inductive.algorithm.Variants.IMf, noise_threshold=0.9)
viz_pn(std_pn_pm4py, std_im_pm4py, std_fm_pm4py, pn_file_name_base)
no_use = viz_3(std_pn_pm4py, std_im_pm4py, std_fm_pm4py)

/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/

In [66]:
# consider the month

import pandas as pd
import os

sample_df = pd.read_csv('/Users/qingtan/Desktop/role_mining_code/sample_new_events.csv')
sample_df["role_activity"] = sample_df["group_role_id"] + "|" + sample_df["activity_type"]

output_dir = "logs_by_agent"
os.makedirs(output_dir, exist_ok=True)

# Split by agent
for agent, sub_df in sample_df.groupby("agent_id"):
    # Sort by timestamp
    sub_df["timestamp"] = pd.to_datetime(sub_df["timestamp"], errors="coerce")
    sub_df = sub_df.sort_values("timestamp").copy()
    
    # --- case id (weekly, sequential labels c1, c2, ...) ---
    # ensure timestamp is datetime
    # df['timestamp'] = pd.to_datetime(df['timestamp'])
    # calculate week start (Monday) for each event
    sub_df['week_start'] = sub_df['timestamp'] - pd.to_timedelta(sub_df['timestamp'].dt.weekday, unit='d')

    # temporary case key: agent + month (YYYY-MM)
    sub_df['temp_case_key'] = sub_df['timestamp'].dt.strftime('%Y-%m')

    # map each unique temp_case_key to c1, c2, c3, ...
    unique_cases = sub_df['temp_case_key'].unique()
    case_map = {key: f'c{i+1}' for i, key in enumerate(unique_cases)}
    sub_df['case_id'] = sub_df['temp_case_key'].map(case_map)
    
    # Save to CSV
    filename = os.path.join(output_dir, f"month_{agent}.csv")
    sub_df.to_csv(filename, index=False)
    print(f"Saved {filename}")

Saved logs_by_agent/month_r11744364.csv
Saved logs_by_agent/month_r2670601.csv
Saved logs_by_agent/month_r3273854.csv
Saved logs_by_agent/month_r560462.csv
Saved logs_by_agent/month_r560872.csv
Saved logs_by_agent/month_r560890.csv
Saved logs_by_agent/month_r560912.csv
Saved logs_by_agent/month_r560925.csv
Saved logs_by_agent/month_r560950.csv
Saved logs_by_agent/month_r9264148.csv


In [67]:
# analyze agent month behaviors

import pm4py
from pm4py.convert import convert_to_petri_net
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import networkx as nx
from discovery_AM.new_picture import graphviz_visualization_IM as viz_3
import importlib


csv_path = '/Users/qingtan/Desktop/role_mining_code/logs_by_agent/agent_month_behaviors/month_r2670601.csv'
agent_name = csv_path.split("/month_")[1].split(".csv")[0]
log_df_test = pd.read_csv(csv_path)
log_df = pm4py.format_dataframe(log_df_test, case_id='case_id', activity_key='role_activity', timestamp_key='timestamp')
log_pm4py = pm4py.convert_to_event_log(log_df)

def discover_pn_imf_pm4py(log_pm4py,im_variant,noise_threshold=0.0):
    # possible values for im_variant:
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMd
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IMf
    #    pm4py.algo.discovery.inductive.algorithm.Variants.IM_CLEAN (used in pm4py.discover_petri_net_inductive)
    parameters = pm4py.utils.get_properties(log_pm4py)
    # parameters[im_variant.value.Parameters.NOISE_THRESHOLD] = noise_threshold (change)
    parameters[pm4py.algo.discovery.inductive.variants.imf.IMFParameters.NOISE_THRESHOLD] = noise_threshold
    # pt = pm4py.algo.discovery.inductive.algorithm.apply_tree(log_pm4py, variant=im_variant, parameters=parameters) (change)
    pt = pm4py.algo.discovery.inductive.algorithm.apply(log_pm4py, variant=im_variant, parameters=parameters)
    # from pm4py.convert import convert_to_petri_net
    # convert_to_petri_net(pt) will generate three parts, which are PetriNet, the initial marking and the final marking
    return convert_to_petri_net(pt)


def viz_pn(pn_pm4py,pn_im_pm4py,pn_fm_pm4py,pn_file_name_base):
    # pn_nx = pn_pm4py2nx(pn_pm4py,initial_marking=pn_im_pm4py,final_marking=pn_fm_pm4py)
    # nx.write_gml(pn_nx,pn_file_name_base+".gml")
    gviz = pn_visualizer.apply(pn_pm4py, pn_im_pm4py, pn_fm_pm4py, parameters={pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "pdf"}, variant=pn_visualizer.Variants.FREQUENCY)
    pn_visualizer.save(gviz, pn_file_name_base+".pdf")


pn_file_name_base = f'/Users/qingtan/Desktop/role_mining_code/sample_log_results/{agent_name}'
std_pn_pm4py, std_im_pm4py, std_fm_pm4py = discover_pn_imf_pm4py(log_pm4py, pm4py.algo.discovery.inductive.algorithm.Variants.IMf, noise_threshold=0.3)
viz_pn(std_pn_pm4py, std_im_pm4py, std_fm_pm4py, pn_file_name_base)
no_use = viz_3(std_pn_pm4py, std_im_pm4py, std_fm_pm4py)

/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/qingtan/anaconda3/lib/python3.11/site-packages/pm4py/objects/log/util/dataframe_utils.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/Users/